[Model delegation](https://dev.writer.com/api-guides/model-delegation) allows you to delegate specific tasks to an domain-specific LLM during a chat. It uses a predefined [LLM tool](https://dev.writer.com/api-guides/model-delegation#tool-object) that you can pass when chatting with a general model.

This cookbook shows how to use model delegation to refer medical questions to the [Palmyra-Med](https://dev.writer.com/models/palmyra-med) model during a chat.

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)

## Setup

Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

You've now initialized the Writer client and can use it to create a chat with an AI model.

First, define the tools you want to provide to the LLM. In this example, you [define a tool](https://dev.writer.com/api-guides/model-delegation#tool-object) to use [Palmyra-Med](https://dev.writer.com/models/palmyra-med) to answer medical questions.

The `type` field for the tool is `llm`. In the `function` field, the `model` is the ID of the [Palmyra model](https://dev.writer.com/home/models) you want to use. The `description` field describes when the tool should be used.

> **Best practice:** The `description` field should indicate that the tool is an LLM. It should specify what the LLM is specialised in and when the tool should be used. The example below is a good description for a tool that invokes a medical LLM.

In [ ]:
tools = [
    {
        "type": "llm",
        "function": {
            "description": "A function that will invoke the llm identified by the given model, "
                "specialised in answering medical queries. Any user request asking for "
                "medical advice should use this tool.",
            "model": "palmyra-med",
        }
    }
]

Once you've defined the LLM tool, you can provide them to the LLM in the `tools` parameter when you create a chat.

In this example, you are asking a medical question, so the general chat model will delegate the task to the medical model.

In [ ]:
response = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": "I feel pain in my lower knee after running. How do I treat it?"}],
    tools=tools,
    tool_choice="auto"
)

print(response.choices[0].message.content)

The output includes the response from the medical model. 

The `response.choices[0].message.llm_data` field indicates that the response comes from an LLM tool call. It shows the model that was used to generate the response and the prompt.

In [ ]:
print(response.choices[0].message.llm_data)

## Streaming

The Writer client also supports streaming responses. In this example, you stream the chat response and collect the output text as it streams.

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": "I feel pain in my lower knee after running. How do I treat it?"}],
    tools=tools,
    tool_choice="auto",
    stream=True
)

output_text = ""

for chunk in response_stream:
    if chunk.choices[0].delta.content:
        output_text += chunk.choices[0].delta.content
    else:
        continue

print(output_text)